<a href="https://colab.research.google.com/github/hatahataDev/SDGsproject/blob/master/STI_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#上から順に実行していってください．
#最後のコードは１時間ほどかかります．すべての処理が終わると予測データのＣＳＶが出力されます．

from google.colab import drive # グーグルドライブをマウントさせるコード。
drive.mount('/content/drive')
data_dir="drive/My Drive/*/STI2_v2.csv" # CSVファイルのパス　学習データ
predict_file="drive/My Drive/*/4th STI-Gigaku.csv" # CSVファイル 予測したいデータ

In [2]:
# https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0 参考記事
# https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0
# ネット回線ないとコード動きません。

def data_change(sdgs_input_data):
  import os
  import csv
  import nltk
  from sklearn.model_selection import train_test_split

  nltk.download('punkt') # nltkを使うためのソフト群
  nltk.download('tagsets')
  nltk.download('averaged_perceptron_tagger')

  csv_file = open(data_dir,'r') # 学習データ
  p_n_list = sdgs_input_data
  # p_csv_file= open(predict_dir,"r") # 予測したいデータ
  """
  csvファイルは左から目標番号、タイトル、アブスト、結論の順で入れること。
  """
  p_list,t_list,a_list = [],[],[]
  # c_list= []

  delete= ["a","%","of","in","on","to","for","are","at","[","]"] # 関連性のない語を集めた配列

  # p_n_list=[]#予測したいデータのテキストバッグ

  # for row in csv.reader(p_csv_file):
  # p_n_list.append(row[1])
  # p_n_list.append(row[2])

  x, y = [], []
  count = 0

  for row in csv.reader(csv_file):
    # 以下リストを作成
      p_list.append(row[0]) # 取得したい列番号を指定（0始まり）#目標番号をすべて取得
      t_list.append(row[1]) # タイトルをすべて取得
      a_list.append(row[2]) # アブストをすべて取得
      # c_list.append(row[3]) # 結論も入れる

  del p_list[0] # 0番目を削除
  del t_list[0] # 0番目は削除
  del a_list[0] # 0番目は削除。これで_list[i]でそれぞれの列を抜き出すことができる。
  # del c_list[0]

  # 以下３行　予測したいデータの形態素解析
  # p_n_list=p_n_list[0]+p_n_list[1] 
  p_n_words = nltk.word_tokenize(p_n_list)
  p_n_wards=nltk.pos_tag(p_n_words)# [("単語","品詞")....]の２次元リストをここで生成


  p_result = [] # 名詞の数字配列を入れるp_resultを定義
  p_N_dic = [] # 名詞のstring型配列を定義
  # 以下名詞を抜き出しN_dicに入れ込み
  for word,part in p_n_wards: # wordに単語、partに品詞が入っている→このforはタイトル
      if part=="NN" or part=="NNP" or part=="VBP": # 以下if文は名詞のみをN_dicというリストに格納
        if not word.lower() in delete: # さらにdeleteリストに入っている意味のない名詞を削除
          sword=word.lower()
          p_N_dic.append(sword)

  Cdic=[] # 各ゴールの単語をすべて入れる。推薦単語用.Cdic[1]→1番目の単語すべてを返す

  for i in range(18): # それぞれのゴールの中身を内包した収納配列を生成
    a=[]
    Cdic.append(a) # [[],[],[]......18個]この中に単語をすべて入れていく

  dic={} # 辞書で定義
  for i,p in enumerate(p_list): # p_listのiをカウンタ変数,ｐを正解ラベル
    j=i+1 # p_list,a_listなどは0番目が存在しないため無理やりjをカウンター変数に置き換え
    if j>=len(p_list): # +1してるから最後の一つだけ存在しないjが入るためここで分岐処理　t_list[j_max]は存在しないためエラー
      break
    N_dic=[] # ここで名詞リストは初期化
    # print(t_list[i])
    t_string = t_list[i]
    t_words = nltk.word_tokenize(t_string)
    t_wards=nltk.pos_tag(t_words)# [("単語","品詞")....]の２次元リストをここで生成
    # print(t_wards)
    a_string = a_list[i]
    a_words = nltk.word_tokenize(a_string)
    a_wards=nltk.pos_tag(a_words)
    """
    c_string = c_list[j]
    c_words = nltk.word_tokenize(c_string)
    c_wards=nltk.pos_tag(c_words)
    """
    for word,part in t_wards: # wordに単語、partに品詞が入っている→このforはタイトル
      if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": # 以下if文は名詞のみをN_dicというリストに格納
        if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
          sword=word.lower() # 小文字に変換
          N_dic.append(sword)
          Cdic[int(p)].extend([sword]) # swordはstr型なので無理やりリストに変換 extendはlist型しか入らない
    for word,part in a_wards: # wordに単語、partに品詞が入っている→このforはアブスト
      if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": # 以下if文は名詞のみをN_dicというリストに格納
        if not word.lower() in delete:
          sword=word.lower()
          N_dic.append(sword)
          Cdic[int(p)].extend([sword]) # pはstrなのでキャスト
    """
    for word,part in c_wards: #wordに単語、partに品詞が入っている→このforはアブスト
      if part=="NN": #以下if文は名詞のみをN_dicというリストに格納
        N_dic.append(word)
      if part=="NNP":
        N_dic.append(word)
    """
  # print(N_dic)# すべてN_dicにまとめてしまう。

    result=[]
    for word in N_dic:
      word = word.strip()  
      if word == "": continue
      if not word in dic: # 未登録の場合
        dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
        num = count
        count +=1
      # print(num,word)  # 数字と単語を表示
      else:
        num=dic[word] # 数字を辞書で調べる
      result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
    x.append(result)  # リストを配列 ｘ に追加 x = [ [ 10, 11, 12, 3, 8, 13,  … 38 ] , [ 39, 40, 12, 16, 19, … ,56 ] , ….  ,  [ 1200, 1504, 3, 15, … 3300 ] ] 
    p_p=int(p)  # pは文字型で入っているのでキャスト
    y.append(p_p)  # 正解ラベルを配列 y に追加 y = [ 0, 0, 0, 0, … ,8, 8, 8, 8 ] 

  # 以下のfor 予測したいデータの名詞配列の数値化 or 辞書の生成
  for word in p_N_dic:
      word = word.strip()  
      if word == "": continue
      if not word in dic: # 未登録の場合
        dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
        num = count
        count +=1
      # print(num,word)  # 数字と単語を表示
      else:
        num=dic[word] # 数字を辞書で調べる
      p_result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している

  return x,y,p_result,dic 
  #print(x)
  #print(y)

In [3]:
#学習機
def learning_predicting(x,y,p_result,dic):
  import numpy as np
  import keras
  import glob
  from keras.models import Sequential
  from keras.layers import Dense, Dropout, Activation
  from keras.preprocessing.text import Tokenizer
  from sklearn.model_selection import train_test_split ### 追加
  from collections import Counter #カウントするメソッドを使うためのモジュール

  max_words = len(dic)
  batch_size = 2
  epochs = 10

  #x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.0025, random_state=111)
  x_train=x
  y_train=y

  print(len(x_train), 'train sequences')
  #print(len(x_test), 'test sequences')
  
  num_classes = np.max(y_train) +1
  print(num_classes, 'classes')
  
  print('Vectorizing sequence data...')
  tokenizer = Tokenizer(num_words=max_words)
  x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
  #x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

  print('x_train shape:', x_train.shape)
  #print('x_test shape:', x_test.shape)
  
  print('Convert class vector to binary class matrix '
  '(for use with categorical_crossentropy)')
  y_train = keras.utils.to_categorical(y_train, num_classes)
  #y_test = keras.utils.to_categorical(y_test, num_classes)
  print('y_train shape:', y_train.shape)
  #print('y_test shape:', y_test.shape)

  # modelのインスタンス生成
  print('Building model...')
  model = Sequential()

  model.add(Dense(206, input_shape=(max_words,)))
  model.add(Activation("sigmoid"))
  model.add(Dropout(0.2))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))#２層ニューラルネットワーク


  model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']) 
  
  history = model.fit(x_train, y_train,
  batch_size=batch_size,
  epochs=epochs,
  verbose=0)
  #score = model.evaluate(x_test, y_test,
  #batch_size=batch_size, verbose=1)

  #pre=model.predict( x_test, batch_size=batch_size, verbose=1) #それぞれの正解ラベルに対する確率を返す

  #予測データのバイナリー化
  def binary(result):
    arr=np.zeros(len(dic)) #辞書数分のバイナリー配列を生成。すべて０で最初は生成
    for f in result: #予測したい文脈の名詞の番号のところに１を立てる。
      arr[f]=1

    arr=np.array(arr) #numpy配列に変換
    arr.reshape(1,len(dic))
    arrd=np.stack([arr, arr]) #無理やり配列を拡張。model.predictの引数に拡張しないと入れられないため。(2,len(dic))の配列にする
    return arrd
  #print(arrd.shape)

  #print('Test score:', score[0])
  #print('Test accuracy:', score[1])
  #print(pre1)

  #出力関数
  def result(pre):
    result= []
    for i,c in enumerate(pre[0]): #２次元配列を生成
      hum=[c,i]
      result.append(hum)
    result2=sorted(result,reverse=True) #大きい順にソート

    j=0
    for k,l in enumerate(result2):
      if l[1]==0:
        j=1
        continue
      if j==1:
        print(str(k)+"番目に関連が高いのは"+str(l[1])+"の目標で,関連度は"+str(l[0]*100)+"％です。")
      else:
        print(str(k+1)+"番目に関連が高いのは"+str(l[1])+"の目標で,関連度は"+str(l[0]*100)+"％です")
    return result2
  bin_pre=binary(p_result)
  pre1=model.predict( bin_pre, batch_size=batch_size, verbose=1)
  current_result=result(pre1)
  return pre1[0]#current_result

  #推薦単語アルゴリズム
  def renumbering(new_word,p_result): #推薦単語をすべて包括した場合の予測 任意の単語を追加して単語を数値化
    jaka=[]
    for word in new_word:
        word = word.strip()  
        if word == "": continue
        if not word in dic: # 未登録の場合
          dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
          num = count
          count +=1
        #print(num,word)  # 数字と単語を表示
        else:
          num=dic[word] # 数字を辞書で調べる
          jaka.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
    return p_result+jaka

  """
  G=Counter(Cdic[current_result[0][1]]).most_common() #それぞれのゴール単語を包括した辞書を予測論文の１番関連度が高いゴールの単語を多い順に返してくれる。ここで型をリストに変えてくれる
  print(G[0])
  new_words=[] #表示させる推薦単語
  for i,k in enumerate(G):
    if i==10: #上位の単語をいくつ調べるかの設定。
      break
    if k[0] in p_N_dic: #その論文に推薦単語がある場合処理をスキップ
      continue
    print("他に包括したほうが良い単語は"+k[0]+"です。\n") 
    new_words.append(k[0])
  #G[0][0] [出現単語][出現回数]

  new_predict=renumbering(new_words,p_result)
  repredict=binary(new_predict) #2値化
  pre2=model.predict( repredict, batch_size=batch_size, verbose=1) #予測データ取得
  print("上記の単語を包括した場合の関連度は以下の様になります。")
  result(pre2) #出力
  """
  """
  print("その他のゴールも包括するには以下のような単語を推薦します")
  for i in range(18):
    if i==0:
      continue
    if i==current_result[0][1]:
      continue
    G=Counter(Cdic[i]).most_common()
    print("Goal"+str(i)+"⇩")
    for i,k in enumerate(G):
      if i==10: #上位の単語をいくつ調べるかの設定。
        break
      if k[0] in p_N_dic: #その論文に推薦単語がある場合処理をスキップ
        continue
      print("他に包括したほうが良い単語は"+k[0]+"です。\n") 
  """
  # Plot accuracy
  """
  import matplotlib.pyplot as plt
  acc = history.history["acc"]
  val_acc = history.history["val_acc"]
  epochs = range(1, len(acc) + 1)
  
  plt.plot(epochs, acc, "bo", label = "Training acc" )
  plt.plot(epochs, val_acc, "b", label = "Validation acc")
  plt.title("Training and Validation accuracy")
  plt.legend()
  plt.savefig("acc.png")
  plt.close()
  
  ### plot Confusion Matrix
  import pandas as pd
  import seaborn as sn
  from sklearn.metrics import confusion_matrix
  
  def print_cmx(y_true, y_pred):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
  
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
  
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cmx, annot=True, fmt="d") ### ヒートマップの表示仕様
    plt.title("Confusion Matrix")
    plt.xlabel("predict_classes")
    plt.ylabel("true_classes")
    plt.savefig("c_matrix.png")
    plt.close()

  predict_classes = model.predict_classes(x_test[1:10000,], batch_size=32) ### 予測したラベルを取得
  true_classes = np.argmax(y_test[1:10000],1) ### 実際のラベルを取得
  print(confusion_matrix(true_classes, predict_classes))
  print_cmx(true_classes, predict_classes)
  """
  #[135, 554, 1126, 160, 2992, 4962, 4963, 2493, 4791, 554]

In [ ]:
#データの予測orファイルへの書き込み
import csv

csv_file = open(predict_file,'r') # 学習データ
p_list=[]
t_list=[]
a_list=[]
#c_list=[]
for row in csv.reader(csv_file):
    # 以下リストを作成
      p_list.append(row[0]) # 取得したい列番号を指定（0始まり）#目標番号をすべて取得
      t_list.append(row[2]) # タイトルをすべて取得
      a_list.append(row[3]) # アブストをすべて取得
      #c_list.append(row[3]) # 結論も入れる #追加の行

csv_file.close()
del p_list[0] # 0番目を削除
del t_list[0] # 0番目は削除
del a_list[0] # 0番目は削除。これで_list[i]でそれぞれの列を抜き出すことができる。
#del c_list[0]

learning_list=[]
for i in range(len(t_list)): #learning_list[0]は一番上の論文
  learning_words=t_list[i]+a_list[i] #+c_list[i]
  learning_list.append(learning_words)

predict_lists=[]
""" #alldatalearning
for i,k in enumerate(learning_list):
  Ta=data_change(k)
  predict_num=learning_predicting(Ta[0],Ta[1],Ta[2],Ta[3]) #出力data_changeでデータをもらってlearning_predictingで予測結果をもらう。
  predict_lists.append([p_list[i]])
  #del predict_num[0]
  for f in range(len(predict_num)):
    if f!=0:
      predict_lists[i].append(predict_num[f]*100)
    #predict_lists[i].append(f[0]*100)
""" 
#solodatalearning
for i in range(len(learning_list)):
  Ta=data_change(learning_list[i])
  predict_num=learning_predicting(Ta[0],Ta[1],Ta[2],Ta[3]) #出力data_changeでデータをもらってlearning_predictingで予測結果をもらう。
  predict_lists.append([p_list[i]])
  #del predict_num[0]
  for f in range(len(predict_num)):
    if f!=0:
      predict_lists[i].append(predict_num[f]*100)
    #predict_lists[i].append(f[0]*100)

  
  """
  ranking_check=[]
  for j in predict_num[0:3]:
    ranking_check.append(j[1])

  if int(p_list[i]) in ranking_check:
    predict_lists.append([p_list[i],1])
  else:
    predict_lists.append([p_list[i],0])
  """
with open('some.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
    writer.writerows(predict_lists) # 2次元配列も書き込める